<a href="https://colab.research.google.com/github/lukefong/genai-workshop-public/blob/main/Explain_Cluster_Analysis_with_BQML_and_Vertex_PaLM_API_(go_explain_bqml_cluster_analysis_colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using the Vertex AI PaLM API to explain BQML Clustering

This example demostrates how to use the Vertex AI PaLM API to explain BQML clustering. 

| | |
|----------|-------------|
| Author(s)   | Hussain Chinoy (ghchinoy@) |

[Feedback](https://forms.gle/GX2sCKXDw2ZQ91ne7) is welcome!


Let's log in with Google, load the Vertex AI libraries and restart the runtime

In [ ]:
#!pip install git+https://github.com/googleapis/python-aiplatform.git "shapely<2.0.0" "protobuf==3.19.6"
!pip install google-cloud-aiplatform --upgrade

---

#### ⚠️ Do not forget to click the "RESTART RUNTIME" button above.

---

Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

In [ ]:
import pandas as pd
from typing import Union
import sys
from google.cloud import bigquery

In [ ]:
#@title Setup Project Variables { run: "auto", display-mode: "form" }
project_id = "skilled-circle-347201" #@param {type:"string"}
dataset_name = "bqml_llm" #@param {type:"string"}
model_name = "ecommerce_customer_segment" #@param {type:"string"}
eval_name = model_name + "_eval"
LOCATION = "us-central1"  # @param {type:"string"}
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_name)
dataset = client.create_dataset(dataset_ref)
print(dataset)

## Create a K-means model to cluster ecommerce data

First let's look at our data quickly before we create the model. This query can be run in BigQuery on its own. Try it out!

In [ ]:
query = """
SELECT
  user_id,
  order_id,
  sale_price,
  created_at as order_created_date
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
"""
df = client.query(query).to_dataframe()
df.head()


,user_id,order_id,sale_price,order_created_date
0,27396,34003,2.5,2022-08-29 20:52:51+00:00
1,33356,41399,2.5,2021-09-11 09:25:35+00:00
2,4135,5159,2.5,2021-10-15 09:50:16+00:00
3,47833,59560,2.5,2022-05-31 03:51:50+00:00
4,68859,85914,2.5,2022-12-04 04:40:47+00:00


## `CREATE MODEL` using `KMEANS`

Create a query then start the model creation job, using a python loop to wait for the job to complete. Please note, if you've created the model already, there's no need to rerun this step to create the clustering using model on the data.

In [ ]:
query = """
CREATE OR REPLACE MODEL `bqml_llm.ecommerce_customer_segment`
OPTIONS (
  MODEL_TYPE = "KMEANS",
  NUM_CLUSTERS = 5,
  KMEANS_INIT_METHOD = "KMEANS++",
  STANDARDIZE_FEATURES = TRUE )
AS (
SELECT * EXCEPT (user_id)
FROM (
  SELECT user_id,
    DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) as DATE), day) AS days_since_order, -- RECENCY
    COUNT(order_id) AS count_orders, -- FREQUENCY
    AVG(sale_price) AS avg_spend -- MONETARY
  FROM (
    SELECT user_id,
      order_id,
      sale_price,
      created_at as order_created_date
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
  )
  GROUP BY user_id, order_id
 )
)
""".format(dataset_name, model_name)


In [ ]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [ ]:
print(query)

# this should take under 5 minutes to create the model
run_bq_query(query)

NameError: ignored

Let's take a look at the model's clustering performance, using these metrics - [Davies Bouldin Index](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index) and Mean Squared Distance

In [ ]:
query = """
SELECT *
FROM ML.EVALUATE(MODEL `{0}.{1}`)
""".format(dataset_name, model_name)
run_bq_query(query)


Finished job_id: 0129a191-cdef-40bf-8f73-a72f8a763452


,davies_bouldin_index,mean_squared_distance
0,1.052067,0.976708


Now let's get the cluster (centroid) information

In [ ]:
query = """
SELECT
  CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
  avg_spend as average_spend,
  count_orders as count_of_orders,
  days_since_order
FROM (
  SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
  FROM ML.CENTROIDS(MODEL `{0}.{1}`)
)
PIVOT (
  SUM(value)
  FOR feature IN ('avg_spend',  'count_orders', 'days_since_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)
run_bq_query(query)

Finished job_id: 44a78350-528f-44f3-9c6c-c9295941bc50


,centroid,average_spend,count_of_orders,days_since_order
0,cluster 1,53.60,2.00,384.95
1,cluster 2,228.38,1.14,481.38
2,cluster 3,44.57,1.00,342.27
3,cluster 4,49.00,1.17,881.51
4,cluster 5,58.36,3.49,504.43


Whew! That's a lot of metrics and cluster info. How about we explain this to our colleagues using the magic of LLMs.

In [ ]:
df = client.query(query).to_dataframe()
df.to_string(header=False, index=False)

cluster_info = []
for i, row in df.iterrows():
  cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
    .format(row["centroid"], row["count_of_orders"], row["average_spend"], row["days_since_order"]) )

print(str.join("\n", cluster_info))

cluster 1, average spend $45.77, count of orders per person 1.23, days since last order 850.51
cluster 2, average spend $359.51, count of orders per person 1.16, days since last order 525.65
cluster 3, average spend $136.63, count of orders per person 1.27, days since last order 416.48
cluster 4, average spend $57.69, count of orders per person 3.51, days since last order 497.3
cluster 5, average spend $38.5, count of orders per person 1.21, days since last order 317.57


## Explain with Vertex AI PaLM API

Install the python library and restart the runtime

In [ ]:
from google.cloud import aiplatform
#from google.cloud.aiplatform.private_preview.language_models import TextGenerationModel, ChatModel
from vertexai.language_models._language_models import TextGenerationModel, ChatModel

aiplatform.init(project=project_id, location=LOCATION)

Generate a text prediction

In [ ]:
#from google.cloud.aiplatform.private_preview.language_models import TextGenerationModel
from vertexai.language_models._language_models import TextGenerationModel

model = TextGenerationModel.from_pretrained("text-bison@001")

clusters = str.join("\n", cluster_info)

prompt = f"""
You're a creative brand strategist, given the following clusters, come up with creative brand persona, a catchy title, and next marketing action, explained step by step.

Clusters:
{clusters}

For each Cluster:
* Title:
* Persona:
* Next Marketing Step:
"""

print(model.predict(
    prompt,
    max_output_tokens=1024,
    temperature=0.55,
    top_p=0.8,
    top_k=40,
))

**Cluster 1**

* Title: The Loyal Customers
* Persona: This persona is someone who is loyal to your brand and has been shopping with you for a long time. They are typically older and have a higher income. They are looking for quality products and services that they can trust.
* Next Marketing Step: Send them a personalized email with a special offer or discount. This will show them that you appreciate their loyalty and encourage them to continue shopping with you.

**Cluster 2**

* Title: The High-End Shoppers
* Persona: This persona is someone who is looking for high-end products and services. They are typically younger and have a high income. They are looking for the best possible quality and are willing to pay a premium for it.
* Next Marketing Step: Create a social media campaign that highlights your high-end products and services. This will help you reach a wider audience of potential customers.

**Cluster 3**

* Title: The Value Shoppers
* Persona: This persona is someone who is 

Voila! We've now used k-means clustering to create groups of spenders and explain their profiles.

Sometimes, though, you want a little bit [extra](https://cloud.google.com/blog/transform/prompt-debunking-five-generative-ai-misconceptions).

In [ ]:
from vertexai.language_models._language_models import TextGenerationModel

model = TextGenerationModel.from_pretrained("text-bison-001")

cluster_info = str.join('\n', cluster_info)

prompt = f"""
Pretend you're a creative strategist, analyse the following clusters and come up with \
creative brand persona for each that includes the detail of which Taylor Swift song is \
likely to be their favorite, a summary of how this relates to their purchasing behavior, \
and a witty e-mail headline for marketing campaign targeted to their group.

Clusters:
{cluster_info}
"""

print(model.predict(
    prompt,
    max_output_tokens=1024,
    temperature=0.45,
    top_p=0.8, top_k=40,
))

**Cluster 1**

* **Average age:** 25
* **Average income:** $75,000
* **Location:** Urban
* **Occupation:** Professional
* **Interests:** Fashion, travel, music, art
* **Favorite Taylor Swift song:** "Shake It Off"

This cluster is made up of young, professional women who are interested in fashion, travel, music, and art. They are likely to be active on social media and enjoy following celebrities. They are also likely to be interested in fashion and beauty trends.

**Witty e-mail headline:** "Shake it off with these new Taylor Swift-inspired fashion finds!"

**Body copy:**

Taylor Swift is a fashion icon, and her songs are always on the radio. If you're a fan of Taylor Swift, you'll love these new fashion finds! From clothes to accessories, we've rounded up the best Taylor Swift-inspired fashion items.

**Call to action:**

Shop now and get 20% off your purchase!

**Cluster 2**

* **Average age:** 30
* **Average income:** $100,000
* **Location:** Suburban
* **Occupation:** Stay-at-home